

>## Importing data from Kaggle api



In [18]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [19]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [20]:
import zipfile
zip_ref = zipfile.ZipFile('/content/fake-and-real-news-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [21]:
import pandas as pd
import numpy as np

In [22]:
df_fake = pd.read_csv('/content/Fake.csv')

In [23]:
df_fake['label'] = 0

In [24]:
df_fake.shape

(23481, 5)

In [25]:
df_fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [26]:
df_real = pd.read_csv('/content/True.csv')
df_real['label'] = 1

## Combining both dataframe together

In [27]:
df = pd.concat([df_real , df_fake])

In [28]:
df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [29]:
df.shape

(44898, 5)

In [31]:
df = df[['title','label']]

In [32]:
df.head()

,title,label
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [34]:
df['label'].value_counts()    #### Dataset is balanced

0    23481
1    21417
Name: label, dtype: int64

## Text Preprocessing

In [35]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
ps = PorterStemmer()

In [40]:
corpus =[]
for i in df['title']:
  data = re.sub('[^a-zA-Z0-9\s]','',i)
  data = data.lower()
  data = ''.join(data)
  data = data.split()
  data = [ps.stem(words) for words in data if words not in stopwords.words('english')]
  data = ' '.join(data)
  corpus.append(data)

In [43]:
corpus[0]

'us budget fight loom republican flip fiscal script'

In [49]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense , Dropout , LSTM , Embedding 
from keras.utils import pad_sequences

In [50]:
voc_size = 5000      ## fixing size of our vocab

In [54]:
oh_rep = [one_hot(sent,voc_size) for sent in corpus]    # converting every words in sentence into onehot rep(gives index of true value of that word in vocab)

In [59]:
oh_rep[0]

[3656, 3502, 595, 1913, 2362, 4069, 1790, 2935]

In [64]:
x = pad_sequences(oh_rep , padding = 'post', maxlen = 15)   ## Padding to make input of equal shape for feeding to nerural network

In [65]:
x

array([[3656, 3502,  595, ...,    0,    0,    0],
       [3656, 4843, 2793, ...,    0,    0,    0],
       [1361, 3656, 2362, ...,    0,    0,    0],
       ...,
       [ 668, 3656, 4523, ...,    0,    0,    0],
       [1306, 2051, 2559, ...,    0,    0,    0],
       [1934, 3656, 3933, ...,    0,    0,    0]], dtype=int32)

In [67]:
y = df['label']
y

0        1
1        1
2        1
3        1
4        1
        ..
23476    0
23477    0
23478    0
23479    0
23480    0
Name: label, Length: 44898, dtype: int64

## Spiltting data into train and test

In [68]:
from sklearn.model_selection import train_test_split
x_train , x_test ,y_train ,y_test = train_test_split(x,y,test_size =0.2,random_state =10)

## Making Embedding layer and LSTM Model

In [78]:
model = Sequential()
model.add(Embedding(voc_size , 40 , input_length=15))  ## 40 is number of dimension in which we want to express index of our each word
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1 , activation ='sigmoid'))
model.compile(optimizer='adam',loss = 'binary_crossentropy', metrics =['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 15, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [79]:
x.shape

(44898, 15)

In [80]:
y.shape

(44898,)

## Fitting data and testing model

In [81]:
model.fit(x_train , y_train , validation_data =(x_test,y_test) , epochs =10 , batch_size =64)

Epoch 1/10
562/562 [==============================] - 19s 29ms/step - loss: 0.2247 - accuracy: 0.9080 - val_loss: 0.1759 - val_accuracy: 0.9310
Epoch 2/10
562/562 [==============================] - 6s 10ms/step - loss: 0.1454 - accuracy: 0.9445 - val_loss: 0.1729 - val_accuracy: 0.9341
Epoch 3/10
562/562 [==============================] - 4s 8ms/step - loss: 0.1277 - accuracy: 0.9524 - val_loss: 0.1717 - val_accuracy: 0.9350
Epoch 4/10
562/562 [==============================] - 4s 8ms/step - loss: 0.1111 - accuracy: 0.9590 - val_loss: 0.1584 - val_accuracy: 0.9410
Epoch 5/10
562/562 [==============================] - 5s 8ms/step - loss: 0.0914 - accuracy: 0.9663 - val_loss: 0.1700 - val_accuracy: 0.9414
Epoch 6/10
562/562 [==============================] - 4s 7ms/step - loss: 0.0680 - accuracy: 0.9752 - val_loss: 0.1810 - val_accuracy: 0.9393
Epoch 7/10
562/562 [==============================] - 6s 11ms/step - loss: 0.0488 - accuracy: 0.9823 - val_loss: 0.2078 - val_accuracy: 0.9421
Ep

In [82]:
ypred = model.predict(x_test)

281/281 [==============================] - 1s 2ms/step


In [85]:
ypred = np.where(ypred>0.5 ,1,0)

In [86]:
ypred

array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [87]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , ypred)

0.9363028953229399

In [90]:
from sklearn.metrics import classification_report
print(classification_report(y_test ,ypred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      4714
           1       0.93      0.94      0.93      4266

    accuracy                           0.94      8980
   macro avg       0.94      0.94      0.94      8980
weighted avg       0.94      0.94      0.94      8980



In [92]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test , ypred)

array([[4400,  314],
       [ 258, 4008]])

## Converting model to pickle file

In [93]:
import pickle
with open('model.pkl','wb') as file:
  pickle.dump(model,file)